In [1]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
import os 
import sys 
import warnings 
import random
from pprint import pprint as pp
from dotenv import load_dotenv
import os
from huggingface_hub import whoami, HfFolder

import gc
from tqdm import tqdm

import torch
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss
from torch.utils.data.dataloader import DataLoader

from transformers import BitsAndBytesConfig
from transformers import DataCollatorForLanguageModeling
from transformers import AutoTokenizer, AutoModelForCausalLM 
from transformers import set_seed, Seq2SeqTrainer, LlamaTokenizer

from datasets import Dataset, DatasetDict

from peft import LoraConfig, get_peft_model


c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from utils import *
import json

## Checking whats wrong

In [3]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
import os 
import sys 
import warnings 
import random
from pprint import pprint as pp
from dotenv import load_dotenv
from datetime import date
import re
from tqdm.auto import tqdm
import emoji
import json
from huggingface_hub import whoami, HfFolder
from copy import deepcopy


import gc

import torch
import torch.distributed as dist
from torch.utils.data.distributed import DistributedSampler
import torch.nn as nn
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss, Softmax, KLDivLoss
from torch.utils.data.dataloader import DataLoader

from transformers import BitsAndBytesConfig
from transformers import DataCollatorForLanguageModeling
from transformers import AutoTokenizer, AutoModelForCausalLM 
from transformers import set_seed, Seq2SeqTrainer, LlamaTokenizer

from datasets import Dataset, DatasetDict

from peft import LoraConfig, get_peft_model

from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score


In [4]:
with open("llm_experiments_set_up.json", "r") as f:
    exp_setup = json.load(f)


In [5]:
mode=None
batch_size=16
type_experiment="from_expl_to_impl"
cl_technique="ewc"
model_id = "Models/SmolLM2-360M-Instruct"
training_order=["explicit_hs", "implicit_hs"]
testing_order=["explicit_hs", "implicit_hs"]
batch_size = batch_size
n_epochs = 8
lr = 1e-4
lora_r = 8
exp_setup = exp_setup
mode = None
dataset_path="df_from_exp_to_imp.csv"

### Preparing the data

In [6]:
class CLTechniques:
    """Container for all continual learning techniques"""

    def __init__(self, model, device, technique="none",
                ewc_lambda=1000,
                mem_size=100,
                lwf_lambda=1,
                temperature=2,
                mas_lambda=1000):

        self.model = model
        self.device = device
        self.technique = technique.lower()

        # Initialize selected technique
        if self.technique == "ewc":
            self._init_ewc(ewc_lambda)
        elif self.technique == "agem":
            self._init_agem(mem_size)
        elif self.technique == "lwf":
            self._init_lwf(lwf_lambda, temperature)
        elif self.technique == "mas":
            self._init_mas(mas_lambda)

    def _init_ewc(self, ewc_lambda):
        """Elastic Weight Consolidation"""
        self.ewc_lambda = ewc_lambda
        self.params = {n: p.clone().detach()
                    for n, p in self.model.named_parameters()
                    if p.requires_grad}
        self.fisher = {n: torch.zeros_like(p)
                    for n, p in self.model.named_parameters()
                    if p.requires_grad}

    def _init_agem(self, mem_size):
        """Average Gradient Episodic Memory"""
        self.mem_size = mem_size
        self.memory = []

    def _init_lwf(self, lwf_lambda, temperature):
        """Learning Without Forgetting"""
        self.lwf_lambda = lwf_lambda
        self.temperature = temperature
        self.old_model = None

    def _init_mas(self, mas_lambda):
        """Memory Aware Synapses"""
        self.mas_lambda = mas_lambda
        self.importance = {n: torch.zeros_like(p)
                        for n, p in self.model.named_parameters()
                        if p.requires_grad}
        self.old_params = deepcopy(self.importance)

    def compute_regularization(self, inputs=None):
        """Compute CL regularization term"""
        if self.technique == "ewc":
            penalty = 0
            for n, p in self.model.named_parameters():
                if p.requires_grad:
                    penalty += (self.fisher[n] * (p - self.params[n]).pow(2)).sum()
            return self.ewc_lambda * penalty

        elif self.technique == "lwf" and self.old_model:
            with torch.no_grad():
                logits = inputs['logits']
                actual_inputs = {k:torch.squeeze(v).to(self.device) for k,v in inputs.items() if k != "logits"}
                old_outputs = self.old_model(**actual_inputs)
            return self.lwf_lambda * KLDivLoss(reduction='batchmean')(
                torch.log_softmax(logits/self.temperature, dim=1),
                torch.softmax(old_outputs.logits/self.temperature, dim=1)
            ) * (self.temperature ** 2)

        elif self.technique == "mas":
            penalty = 0
            for n, p in self.model.named_parameters():
                if p.requires_grad:
                    penalty += (self.importance[n] * (p - self.old_params[n]).pow(2)).sum()
            return self.mas_lambda * penalty

        return 0

    def pre_backward(self, inputs=None):
        """Operations before backward pass"""
        if self.technique == "agem" and self.memory:
            # Store current gradient
            self.model.zero_grad()
            for inputs_mem, labels_mem in self.memory:
                inputs_mem = {k:torch.squeeze(v).to(self.device) for k,v in inputs_mem.items()}
                labels_mem = torch.squeeze(labels_mem).to(self.device)
                outputs = self.model(**inputs_mem)
                loss = loss_f(outputs.logits, labels_mem)
                loss.backward()

            self.ref_grad = [p.grad.clone() for p in self.model.parameters() if p.requires_grad] # i think this should be with req grad
            self.model.zero_grad()

    def post_backward(self):
        """Operations after backward pass"""
        if self.technique == "agem" and hasattr(self, 'ref_grad'):
            # Project gradients
            dot_product = sum(torch.sum(p.grad * g_ref)
                        for p, g_ref in zip(self.model.parameters(), self.ref_grad) if p.requires_grad)
            ref_norm = sum(torch.sum(g_ref * g_ref) for g_ref in self.ref_grad)

            if dot_product < 0:  # Negative interference
                scale = dot_product / (ref_norm + 1e-8)
                for p, g_ref in zip(self.model.parameters(), self.ref_grad):
                    if p.grad is not None and p.requires_grad:
                        p.grad -= scale * g_ref

    def post_task_update(self, dataloader=None):
        """Update after each task"""
        if self.technique == "ewc":
            # Compute Fisher information
            self.model.eval()
            for batch in dataloader:
                self.model.zero_grad()
                inputs = {k:torch.squeeze(v).to(self.device) for k, v in batch.items()
                        if k in ['input_ids', 'attention_mask']}
                labels = batch['labels'].to(self.device)

                # outputs = self.model(**inputs, labels=labels)
                outputs = self.model(**inputs)

                loss = loss_f(outputs.logits, labels)
                loss.backward()

                for n, p in self.model.named_parameters():
                    if p.requires_grad and p.grad is not None:
                        self.fisher[n] += p.grad.pow(2) / len(dataloader)

            # Update stored parameters
            self.params = {n: p.clone().detach()
                        for n, p in self.model.named_parameters()
                        if p.requires_grad}

        elif self.technique == "agem":
            # Update memory buffer
            self.memory = []
            for batch in dataloader:
                inputs = {k: torch.squeeze(v).to(self.device) for k, v in batch.items()
                        if k in ['input_ids', 'attention_mask']}
                labels = torch.squeeze(batch['labels']).to(self.device)
                self.memory.append((inputs, labels))
                if len(self.memory) >= self.mem_size:
                    break

        elif self.technique == "lwf":
            # Save model snapshot
            self.old_model = deepcopy(self.model)
            self.old_model.eval()

        elif self.technique == "mas":
            # Update importance weights
            self.model.eval()
            for batch in dataloader:
                self.model.zero_grad()
                inputs = {k: torch.squeeze(v).to(self.device) for k, v in batch.items()
                        if k in ['input_ids', 'attention_mask']}

                outputs = self.model(**inputs)
                # does this need a loss?????????????
                torch.norm(outputs.logits, p=2, dim=1).mean().backward()

                for n, p in self.model.named_parameters():
                    if p.requires_grad and p.grad is not None:
                        self.importance[n] += p.grad.abs() / len(dataloader)

            # Update stored parameters
            self.old_params = {n: p.clone().detach()
                            for n, p in self.model.named_parameters()
                            if p.requires_grad}

class AutoContinualLearner(nn.Module):
    def __init__(self, model_name, device, quantization_config, torch_dtype=torch.bfloat16):
        super().__init__()
        self.device = device
        # self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            quantization_config=quantization_config,
            torch_dtype=torch_dtype
        )
        self.n_initial_params = sum(t.numel() for t in self.model.parameters())
        self.n_trainable_params_initial = sum(t.numel() for t in self.model.parameters() if t.requires_grad)
        self.cl = None

    def init_cl(self, technique, lora_config, **kwargs):
        """Init the continual learning technique"""
        self.model = get_peft_model(self.model, lora_config).to(self.device)
        self.n_params_lora = sum(t.numel() for t in self.model.parameters())
        self.n_trainable_params_lora = sum(t.numel() for t in self.model.parameters() if t.requires_grad)
        self.model.print_trainable_parameters()
        self.cl = CLTechniques(self.model, self.device, technique, **kwargs)
    def forward(self, **kwargs):
        return self.model(**kwargs)


In [7]:
def log_failed_batch(batch):
    
    print("FAILED UNSQUEEZED BATCH")
    if "failed_batches.json" not in list(os.listdir()):
        with open("failed_batches.json", "w") as f:
            json.dump([], f)
    with open("failed_batches.json", "r") as f:
        failed_batches = json.load(f)
    failed_batches.append(batch)
    with open("failed_batches.json", "w") as f:
        json.dump(failed_batches, f)

def log_hf():
    
    load_dotenv("env_vars.env")
    hf_token = os.environ.get("HF_ACCESS_TOKEN")
    HfFolder.save_token(hf_token)
    return print(whoami()["name"])

def setup():
    try:
        dist.init_process_group("nccl")
        # dist.init_process_group("gloo")
        local_rank = int(os.environ["LOCAL_RANK"])
        torch.cuda.set_device(local_rank)
        return local_rank
    except Exception as e:
        print(e)
        print("DISTR TRAINING ALREADY INITIALIZED")
        local_rank = int(os.environ["LOCAL_RANK"])
        torch.cuda.set_device(local_rank)
        return local_rank


def save_results_csv(df, experiment_name, model_id, cl_technique, result_type="specific"):

    cl_technique_clean = cl_technique.replace(" + ", "__")
    id_ = model_id.replace("/", "-") + "_" + cl_technique_clean + "_" + str(date.today())
    id_clean = experiment_name + id_.replace(" ", "_").replace(":", "-").replace(".","-") + result_type + ".csv"
    df.to_csv(id_clean, index=False)
    return print("Saved in path: ", id_clean)

def clean_cl_name(cl_name):

    regex = r'<(?:[\w\.]+)?\.([\w]+) object at'
    matches =   re.findall(regex, cl_name)
    clean_string = " + ".join(matches)
    return clean_string

def clean_metric_name(metric_name):

    reg = r"\s([a-z_1]+)\s"
    match_ = re.search(reg, metric_name)
    clean_str = match_.group().strip()

    return clean_str

def translate_class_to_label(class_):

    translation_dict = {"not_hate": "NOT HATEFUL",
                        "explicit_hate": "HATEFUL",
                        "implicit_hate": "HATEFUL"}

    translated_label = translation_dict[class_]

    return translated_label

def format_message(formatted_prompt, label=True):
    if label:
        messages = [
            {"role": "system", "content": "You are a helpful assistant"},
            {"role": "user", "content": formatted_prompt},
            {"role": "assistant", "content": label}
        ]
    else:
        messages = [
            {"role": "system", "content": "You are a helpful assistant"},
            {"role": "user", "content": formatted_prompt}
        ]
    return messages

base_prompt = """You are a social media content moderator.
INSTRUCTION: The following is a social media message that needs to be classified with the label HATEFUL or NOT HATEFUL.
MESSAGE: {}
OUTPUT AND FORMAT: your output should be just the label."""

def format_prompt(text, base_prompt=base_prompt):

    formatted_prompt = base_prompt.format(text)
    
    return formatted_prompt

def get_probability_distribution(logits):
    probability_dist = Softmax(dim=-1)(logits)
    return probability_dist

loss_fn = CrossEntropyLoss(ignore_index=-100) # ignore the left pad tokens
def loss_f(logits, labels):

    flat_logits = logits.view(-1, logits.size(-1))
    flat_labels = labels.view(-1)

    loss = loss_fn(flat_logits, flat_labels)
    
    return loss

def translate_prediction_to_label(text):
    if "NOT HATEFUL" in text:
        text_clean = text.replace("NOT HATEFUL", "")
        if "HATEFUL" in text_clean or "HATEFUAL" in text_clean:
            return 2
        else:
            return 0
    elif "NOT_HATEFUL" in text:
        text_clean = text.replace("NOT_HATEFUL", "")
        if "HATEFUL" in text_clean or "HATEFUAL" in text_clean:
            return 2
        else: 
            return 0
    elif "HATEFUL" in text:
        text_clean = text.replace("HATEFUL", "")
        if "NOT_HATEFUL" in text_clean or "NOT HATEFUL" in text_clean:
            return 2
        else:
            return 1
    else:
        return 2


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

########################################################## DATA WORK
print("_________________________________")
print("Preapring the Tokenizer")

tokenizer = AutoTokenizer.from_pretrained(model_id + "/Tokenizer")
if tokenizer.pad_token is None and "Llama" in model_id: tokenizer.pad_token = '<|finetune_right_pad_id|>'
elif tokenizer.pad_token is None: tokenizer.pad_token = tokenizer.eos_token
tokenizer.chat_template = open(model_id + "/Tokenizer/chat_template.jinja").read()

# print(tokenizer.chat_template)
# print(tokenizer.apply_chat_template("Hello World", tokenize=False, add_generation_prompt=True, return_tensors="pt"))

def preprocess_and_tokenize(clean_post, label, base_prompt=base_prompt, max_length=512):
    
    prompt_plus_messages = base_prompt.format(clean_post)
    messages = [
            {"role": "system", "content": "You are a helpful assistant"},
            {"role": "user", "content": prompt_plus_messages},
            {"role": "assistant", "content": label.strip("\n")}
        ]

    chat_template = tokenizer.apply_chat_template(messages, tokenize=False, continue_final_message=False, add_special_tokens=False).rstrip()
    input_ids_tokenized = tokenizer(chat_template, return_tensors="pt", add_special_tokens=False, padding="max_length", max_length=max_length)["input_ids"]

    # getting the normal text just to know how much we need to add to the left as -100 and right as pad token
    input_ids_shape = tokenizer(chat_template, return_tensors="pt", add_special_tokens=False, padding=False)["input_ids"]

    # getting the label target to only predict the actual label and ignore the prompt
    labels_tokenized = tokenizer(label + tokenizer.eos_token, add_special_tokens=True, return_tensors="pt")["input_ids"]
    shape = input_ids_shape.shape[1] - labels_tokenized.shape[1]
    zeros = torch.zeros((1, shape), dtype=labels_tokenized.dtype, device=labels_tokenized.device)
    zeros.fill_(-100) # for the cross entropy loss
    labels_left_padded = torch.cat([zeros, labels_tokenized], dim=1)

    eos_n = input_ids_tokenized.shape[1] - labels_left_padded.shape[1]
    eos_n_tensor = torch.zeros((1, eos_n), dtype=labels_tokenized.dtype, device=labels_tokenized.device)
    eos_n_tensor.fill_(tokenizer.encode(tokenizer.pad_token, add_special_tokens=False)[0])
    labels_padded = torch.cat([labels_left_padded, eos_n_tensor], dim=1)

    # print(labels_padded.shape == input_ids_tokenized.shape)

    # shifting because we dont predict the first token
    input_ids_tokenized_left_shifted = input_ids_tokenized[:, :-1]
    labels_tokenized_right_shifted = labels_padded[:, 1:]

    attention_mask = input_ids_tokenized_left_shifted != tokenizer.pad_token_id
    
    return {
        "input_ids": input_ids_tokenized_left_shifted,
        "labels": labels_tokenized_right_shifted,
        "attention_mask": attention_mask
    }

print("----------Preparing the Data-----------------")

print("_________________________________")
print("Loading and filtering the Data")

df = pd.read_csv(dataset_path)

#### Attaching the prompt to the clean post
df["formatted_prompt"] = df["clean_post"].apply(format_prompt)
df["label"] = df["class"].apply(translate_class_to_label)

# ### Turning the Df into a DatasetDict

times_array = list(df["time"].unique())
datasets = []
dataset_names = list(df["task"].unique())

for task in training_order:

    time_ds = []
    for split in df["split"].unique():

        split_df = df[(df["split"] == split) & (df["task"] == task)]
        hf_split = Dataset.from_pandas(split_df)
        time_ds.append(hf_split)
    datasets.append(time_ds)

datasets_test = []
for i, task in enumerate(testing_order):

    split_df = df[(df["split"] == "test") & (df["task"] == task)]
    hf_split = Dataset.from_pandas(split_df)
    datasets_test.append({testing_order[i]: hf_split})

hf_datasets = []

for i, dataset in enumerate(datasets):
    print("dataset")

    hf_ds = DatasetDict({dataset[0]["split"][0]: dataset[0], 
                        dataset[1]["split"][0]: dataset[1],
                        dataset[2]["split"][0]: dataset[2]})
    hf_ds_name = training_order[i]
    hf_datasets.append({hf_ds_name: hf_ds})

hf_datasets_processed = []
for ds in hf_datasets:
    ds_dict = {}
    for task_name, hf_data in ds.items():
        ds_dict[task_name] = {}
        for split in hf_data:
            ds_dict[task_name][split] = hf_data[split].map(preprocess_and_tokenize, input_columns=["clean_post", "label"], batched=False)
    hf_datasets_processed.append(ds_dict)

n_samples_per_ds = [
    len(hf_time["train"])
    for hf_data in hf_datasets
    for task_name, hf_time in hf_data.items() 
]

for ds in hf_datasets_processed:
    for task_name, hf_data in ds.items():
        for split in hf_data:
            hf_data[split].set_format("torch")

cols_to_remove = ["clean_post", "post", "class", "implicit_class", "extra_implicit_class", 
                "target", "implied_statement", "split", "time", "task",
                "formatted_prompt", "label", "__index_level_0__"]

hf_datasets_no_cols = []
for ds in hf_datasets_processed:
    ds_dict = {}
    for task_name, hf_data in ds.items():
        ds_dict[task_name] = {}
        for split in hf_data:
            if split != "test":
                ds_dict[task_name][split] = hf_data[split].remove_columns(cols_to_remove)
    hf_datasets_no_cols.append(ds_dict)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

_________________________________
Preapring the Tokenizer
----------Preparing the Data-----------------
_________________________________
Loading and filtering the Data
dataset
dataset


Map: 100%|██████████| 1020/1020 [00:01<00:00, 751.50 examples/s]


In [9]:
data_loaders = []
for ds in hf_datasets_no_cols:
    ds_dict = {}
    for task_name, hf_data in ds.items():
        ds_dict[task_name] = {}
        for split in hf_data:
            if split != "test":
                data_loader = DataLoader(ds[task_name][split], collate_fn=data_collator, batch_size=batch_size)
                ds_dict[task_name][split] = data_loader
    data_loaders.append(ds_dict)

for i, ds in enumerate(hf_datasets):
    for task_name, hf_data in ds.items():
        for split in hf_data:
            if split == "test":
                data_loaders[i]["test"] = hf_data[split]

print("DATA LOADERS AT THE END OF PROCESSING")
pp(data_loaders)
print()
print("TEST DATA AT THE END OF PROCESSING")
pp(datasets_test)
print()


DATA LOADERS AT THE END OF PROCESSING
[{'explicit_hs': {'train': <torch.utils.data.dataloader.DataLoader object at 0x000001A9CA23C770>,
                  'validation': <torch.utils.data.dataloader.DataLoader object at 0x000001A9BF9865A0>},
  'test': Dataset({
    features: ['clean_post', 'post', 'class', 'implicit_class', 'extra_implicit_class', 'target', 'implied_statement', 'split', 'time', 'task', 'formatted_prompt', 'label', '__index_level_0__'],
    num_rows: 720
})},
 {'implicit_hs': {'train': <torch.utils.data.dataloader.DataLoader object at 0x000001A9CA23C3B0>,
                  'validation': <torch.utils.data.dataloader.DataLoader object at 0x000001A9CA23CD40>},
  'test': Dataset({
    features: ['clean_post', 'post', 'class', 'implicit_class', 'extra_implicit_class', 'target', 'implied_statement', 'split', 'time', 'task', 'formatted_prompt', 'label', '__index_level_0__'],
    num_rows: 1020
})}]

TEST DATA AT THE END OF PROCESSING
[{'explicit_hs': Dataset({
    features: ['cl

## Preparing the Model

In [11]:
epochs_array = []

for i in range(len(training_order)):
    epochs_array.append(n_epochs)

ks_array = None

In [12]:
print("_________________________________")
print("Loading the model and model config with LoRA and 4-bit quantization nf4")

bnb_config = BitsAndBytesConfig(  
                                load_in_4bit= True,
                                bnb_4bit_quant_type= "nf4",
                                bnb_4bit_compute_dtype= torch.bfloat16,
                                bnb_4bit_use_double_quant= True,
                            )

lora_alpha = lora_r*2
config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    task_type="CAUSAL_LM",
    lora_dropout=0.1,
    bias="none",
)

if cl_technique in ["ewc", "agem", "lwf", "mas"]:
    cl_hyperparams = {
    "ewc": {"ewc_lambda":1500},
    "agem": {"mem_size":100},
    "lwf": {"lwf_lambda":1,
            "temperature":2},
    "mas": {"mas_lambda":1000}
    }

    cl_params = cl_hyperparams[cl_technique]
    hyper_param_str = "=".join([str(k) + "-" + str(v) for k, v in cl_params.items()])

    model = AutoContinualLearner(model_id + "/Model", device, bnb_config)
    model.init_cl(technique=cl_technique, lora_config=config, **cl_params)

else:
    cl_params = {"NA": "NA"}
    hyper_param_str = "NA"

    model = AutoContinualLearner(model_id + "/Model", device, bnb_config)
    model.init_cl(technique=cl_technique, lora_config=config)

n_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
optimizer = AdamW((param for param in model.parameters() if param.requires_grad), lr=lr)

print("_________________________________")

hf_datasets = data_loaders

zero_testing_order = [dataset for dataset in testing_order if dataset not in training_order]

results = {
            "model": model_id,
            "training_order": training_order,
            "testing_order": testing_order,
            "zero_testing":zero_testing_order,
            "epochs": epochs_array,
            "exp_setup": exp_setup,
            "cl_technique": cl_technique,
            "type_experiment": type_experiment,
            "batch_size": batch_size,
            "learning_rate": lr,
            "results": []
            }

print("CONTINUAL LEARNING EXPERIMENT SET UP")
for k, v in results.items():
    if k != "exp_setup":
        print(k, ":\t", v)
        print()

data_loaders_train = []
data_loaders_val = []
test_datasets = []
for time, ds in enumerate(training_order):
    # print("training order")
    # print(training_order)
    # print("ds")
    # print(ds)
    # print("hf_datasets")
    # print(hf_datasets[time].keys())
    data_loaders_train.append(hf_datasets[time][ds]["train"])
    data_loaders_val.append(hf_datasets[time][ds]["validation"])
    test_datasets.append({"test":hf_datasets[time]["test"]})
# print("TEST DATASETS BEFORE STARTING THE EXPERIENCE")
# print(test_datasets)

test_results = []
train_results = []

n_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
optimizer = AdamW((param for param in model.parameters() if param.requires_grad), lr=lr)

print("_________________________________")

hf_datasets = data_loaders



_________________________________
Loading the model and model config with LoRA and 4-bit quantization nf4
trainable params: 1,638,400 || all params: 363,459,520 || trainable%: 0.4508
_________________________________
CONTINUAL LEARNING EXPERIMENT SET UP
model :	 Models/SmolLM2-360M-Instruct

training_order :	 ['explicit_hs', 'implicit_hs']

testing_order :	 ['explicit_hs', 'implicit_hs']

zero_testing :	 []

epochs :	 [8, 8]

cl_technique :	 ewc

type_experiment :	 from_expl_to_impl

batch_size :	 16

learning_rate :	 0.0001

results :	 []

_________________________________


In [ ]:
for time, current_training_dataset in enumerate(data_loaders_train): # current tr_ds is a string!!
    print("------------------Starting Experience----------------")
    print(f"------------------TIME {time}------------------------")
    print()
    # torch.cuda.ipc_collect()
    # torch.cuda.empty_cache()
    n_epochs = epochs_array[time]
    if ks_array != None:
        n_samples = ks_array[time]
    else:
        n_samples = n_samples_per_ds[time]

    current_dataset_name = training_order[time]
    current_testing_dataset = testing_order[time]

    print(f"Epochs in the current time: {n_epochs}\nNumber of training samples: {n_samples}\nCurrent Dataset: {current_dataset_name}")
    print()

    train_loader=data_loaders_train[time]
    validation_loader=data_loaders_val[time]


    print("_________________________________")
    print("Training the model")
    print()

    global_training_losses = []
    global_validation_losses = []
    current_testing_dataset=testing_order[time]

    model.train()
    with torch.amp.autocast('cuda', dtype=torch.float32):
    # for task in tasks/dataset - train, eval
        for epoch in tqdm(range(n_epochs)):
            if epoch > 0:
                continue
            torch.cuda.empty_cache()
            gc.collect()

            epoch_validation_losses = []
            train_losses = []

            print("Epoch: ", epoch)

            for i, batch in enumerate(tqdm(train_loader)):

                torch.cuda.empty_cache()
                gc.collect()
                batch_unsqueezed = batch
                # print("\tBatch: ", i)
                batch = {k:torch.squeeze(v).to(device) for k,v in batch.items()}

                # print("Squeezed Batch")
                for k, v in batch.items():
                    if v.shape[0] != batch_size:
                        print(f"{k}: {v.shape}")
                        print()
                        continue
                    


                # print("Unsqueezed Batch")
                # for k, v in batch_unsqueezed.items():
                #     print(f"{k}: {v.shape}")

                # print(batch["input_ids"].shape)
                # print(batch["attention_mask"].shape)
                # print(batch["labels"].shape)
                try:
                    output = model.model(**batch)
                    # print(output)
                    logits = output.logits
                    # print("Shape Logits")
                    # print(logits.shape)
                    # print("Shape Labels")
                    # print(batch["labels"].shape)
                    loss = loss_f(logits, batch["labels"])

                    if i == 0:
                        print("Checking that the model type is the continual learner to do the cls")
                        print(type(model))
                        print(type(model.cl))
                        print(model.cl)
                        # print(dir(model.module))
                    if model.cl:
                        batch['logits'] = logits  # needed for LwF
                        loss += modelodule.cl.compute_regularization(batch)
                        model.cl.pre_backward(batch)
                    # print("CL regularization and backward computed")

                    loss.backward()


                    # needed agem (A-GEM)
                    if model.module.cl:
                        model.module.cl.post_backward()
                    # print("CL post backward computed")

                    optimizer.step()
                    optimizer.zero_grad()

                    train_losses.append(loss.detach().item())

                except Exception as e:

                    print()
                    print(e)
                    print("Switching Batch Size to Unsqueezed")

                #     output = model.module.model(**batch_unsqueezed)
                #     # print(output)
                #     logits = output.logits
                #     # print("Shape Logits")
                #     # print(logits.shape)
                #     # print("Shape Labels")
                #     # print(batch_unsqueezed["labels"].shape)
                #     loss = loss_f(logits, batch_unsqueezed["labels"])
                #     # print(dir(model.module))
                #     if model.module.cl:
                #         batch_unsqueezed['logits'] = logits  # needed for LwF
                #         loss += model.module.cl.compute_regularization(batch_unsqueezed)
                #         model.module.cl.pre_backward(batch_unsqueezed)
                #     print("CL regularization and backward computed")

                #     loss.backward()


                #     # needed agem (A-GEM)
                #     if model.module.cl:
                #         model.module.cl.post_backward()
                #     print("CL post backward computed")

                #     optimizer.step()
                #     optimizer.zero_grad()

                #     train_losses.append(loss.detach().item())

            # epoch_loss = sum(train_losses) / len(train_losses) # loss on current device

            # epoch_loss_tensor = torch.tensor(epoch_loss, device=device)

            # print(f"Epoch Loss: {epoch_loss_tensor.item()}")
            # global_training_losses.append(epoch_loss_tensor.item())

        model.eval()
        with torch.amp.autocast(device_type="cuda", dtype=torch.float32):
            with torch.no_grad():

                print("_________________________________")
                print("Validating the model")
                print()
                torch.cuda.empty_cache()
                gc.collect()

                val_losses = [] # val loss for each batch

                for i, batch in enumerate(tqdm(validation_loader)):
                    
                    # batch.to(device)
                    batch_unsqueezed = batch
                    # print("\tBatch: ", i)
                    batch = {k:torch.squeeze(v).to(device) for k,v in batch.items()}

                    # print("Squeezed Batch")
                    for k, v in batch.items():
                        if v.shape[0] != batch_size:
                            print(f"{k}: {v.shape}")
                            print()
                            continue

                    # print("Unsqueezed Batch")
                    # for k, v in batch_unsqueezed.items():
                    #     print(f"{k}: {v.shape}")

                    try:
                        output = model.model(**batch)
                        # print(output)
                        logits = output.logits
                        # print("Shape Logits")
                        # print(logits.shape)
                        # print("Shape Labels")
                        # print(batch["labels"].shape)
                        val_loss = loss_f(logits, batch["labels"])
                        val_losses.append(val_loss.detach().item())

                    except Exception as e:
                        print("Switching Batch Size to unsqueezed")
                #         output = model.model(**batch_unsqueezed)
                #         logits = output.logits
                #         val_loss = loss_f(logits, batch_unsqueezed["labels"])

                #         val_losses.append(val_loss.detach().item())

                        print()
                        print(e)
                
                # val_loss_epoch = sum(val_losses)/len(val_losses)
                # val_loss_tensor = torch.tensor(val_loss_epoch, device=device)

                # print("_________________________________")
                # print("Validation completed")
                # print()
                # print(f"Validation Loss: {val_loss_tensor.item()}")
                # print("_________________________________")

                # val_loss = val_loss_tensor.item()
                # epoch_validation_losses.append(val_loss)
                # global_validation_losses.append(val_loss)

        # print()
        # print("---------------------TRAINING ENDED---------------")
        # print("Final Training Losses:", global_training_losses)
        # print("Final Validation Losses:", global_validation_losses)

        # if model.module.cl:
        #     model.module.cl.post_task_update(train_loader)

        # print("-----------POST TRAINING CL UPDATES COMPLETED---------")

------------------Starting Experience----------------
------------------TIME 0------------------------

Epochs in the current time: 8
Number of training samples: 5752
Current Dataset: explicit_hs

_________________________________
Training the model



  0%|          | 0/8 [00:00<?, ?it/s]

Epoch:  0



CUDA out of memory. Tried to allocate 30.00 MiB. GPU 0 has a total capacity of 4.00 GiB of which 0 bytes is free. Of the allocated memory 10.45 GiB is allocated by PyTorch, and 188.98 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
Switching Batch Size to Unsqueezed


## ----------------------------------------------------------------------------

In [ ]:
def log_hf():
    
    load_dotenv("env_vars.env")
    hf_token = os.environ.get("HF_ACCESS_TOKEN")
    HfFolder.save_token(hf_token)
    return print(whoami()["name"])

def setup():
    dist.init_process_group("nccl")
    local_rank = int(os.environ["LOCAL_RANK"])
    torch.cuda.set_device(local_rank)
    return local_rank

def save_results_csv(df, experiment_name, model_id, cl_technique, result_type="specific"):

    cl_technique_clean = cl_technique.replace(" + ", "__")
    id_ = model_id.replace("/", "-") + "_" + cl_technique_clean + "_" + str(date.today())
    id_clean = experiment_name + id_.replace(" ", "_").replace(":", "-").replace(".","-") + result_type + ".csv"
    df.to_csv(id_clean, index=False)
    return print("Saved in path: ", id_clean)

def clean_cl_name(cl_name):

    regex = r'<(?:[\w\.]+)?\.([\w]+) object at'
    matches =   re.findall(regex, cl_name)
    clean_string = " + ".join(matches)
    return clean_string

def clean_metric_name(metric_name):

    reg = r"\s([a-z_1]+)\s"
    match_ = re.search(reg, metric_name)
    clean_str = match_.group().strip()

    return clean_str

def translate_class_to_label(class_):

    translation_dict = {"not_hate": "NOT HATEFUL",
                        "explicit_hate": "HATEFUL",
                        "implicit_hate": "HATEFUL"}

    translated_label = translation_dict[class_]

    return translated_label

def format_message(formatted_prompt, label=True):
    if label:
        messages = [
            {"role": "system", "content": "You are a helpful assistant"},
            {"role": "user", "content": formatted_prompt},
            {"role": "assistant", "content": label}
        ]
    else:
        messages = [
            {"role": "system", "content": "You are a helpful assistant"},
            {"role": "user", "content": formatted_prompt}
        ]
    return messages

base_prompt = """You are a social media content moderator.
INSTRUCTION: The following is a social media message that needs to be classified with the label HATEFUL or NOT HATEFUL.
MESSAGE: {}
OUTPUT AND FORMAT: your output should be just the label."""

def format_prompt(text, base_prompt=base_prompt):

    formatted_prompt = base_prompt.format(text)
    
    return formatted_prompt

def get_probability_distribution(logits):
    probability_dist = Softmax(dim=-1)(logits)
    return probability_dist

loss_fn = CrossEntropyLoss(ignore_index=-100) # ignore the left pad tokens
def loss_f(logits, labels):

    flat_logits = logits.view(-1, logits.size(-1))
    flat_labels = labels.view(-1)

    loss = loss_fn(flat_logits, flat_labels)
    
    return loss

def translate_prediction_to_label(text):
    if "NOT HATEFUL" in text:
        text_clean = text.replace("NOT HATEFUL", "")
        if "HATEFUL" in text_clean or "HATEFUAL" in text_clean:
            return 2
        else:
            return 0
    elif "NOT_HATEFUL" in text:
        text_clean = text.replace("NOT_HATEFUL", "")
        if "HATEFUL" in text_clean or "HATEFUAL" in text_clean:
            return 2
        else: 
            return 0
    else:
        return 1


## Data Stuff

In [ ]:
import torch
import torch.distributed as dist
from torch.utils.data.distributed import DistributedSampler
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss, Softmax
import torch.nn as nn
from torch.utils.data.dataloader import DataLoader

from transformers import BitsAndBytesConfig
from transformers import DataCollatorForLanguageModeling
from transformers import AutoTokenizer, AutoModelForCausalLM 
from transformers import set_seed, Seq2SeqTrainer, LlamaTokenizer

from datasets import Dataset, DatasetDict

from peft import LoraConfig, get_peft_model

from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score


In [ ]:
model_id = "Models/SmolLM2-360M-Instruct"

In [ ]:
bnb_config = BitsAndBytesConfig(  
                                    load_in_4bit= True,
                                    bnb_4bit_quant_type= "nf4",
                                    bnb_4bit_compute_dtype= torch.bfloat16,
                                    bnb_4bit_use_double_quant= True,
                                )
                                
lora_r = 8
lora_alpha = lora_r*2
config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    task_type="CAUSAL_LM",
    lora_dropout=0.1,
    bias="none",
    )


In [ ]:
class CLTechniques:
    """Container for all continual learning techniques"""

    def __init__(self, model, device, technique="none",
                ewc_lambda=1000,
                mem_size=100,
                lwf_lambda=1,
                temperature=2,
                mas_lambda=1000):

        self.model = model
        self.device = device
        self.technique = technique.lower()

        # Initialize selected technique
        if self.technique == "ewc":
            self._init_ewc(ewc_lambda)
        elif self.technique == "agem":
            self._init_agem(mem_size)
        elif self.technique == "lwf":
            self._init_lwf(lwf_lambda, temperature)
        elif self.technique == "mas":
            self._init_mas(mas_lambda)

    def _init_ewc(self, ewc_lambda):
        """Elastic Weight Consolidation"""
        self.ewc_lambda = ewc_lambda
        self.params = {n: p.clone().detach()
                    for n, p in self.model.named_parameters()
                    if p.requires_grad}
        self.fisher = {n: torch.zeros_like(p)
                    for n, p in self.model.named_parameters()
                    if p.requires_grad}

    def _init_agem(self, mem_size):
        """Average Gradient Episodic Memory"""
        self.mem_size = mem_size
        self.memory = []

    def _init_lwf(self, lwf_lambda, temperature):
        """Learning Without Forgetting"""
        self.lwf_lambda = lwf_lambda
        self.temperature = temperature
        self.old_model = None

    def _init_mas(self, mas_lambda):
        """Memory Aware Synapses"""
        self.mas_lambda = mas_lambda
        self.importance = {n: torch.zeros_like(p)
                        for n, p in self.model.named_parameters()
                        if p.requires_grad}
        self.old_params = deepcopy(self.importance)

    def compute_regularization(self, inputs=None):
        """Compute CL regularization term"""
        if self.technique == "ewc":
            penalty = 0
            for n, p in self.model.named_parameters():
                if p.requires_grad:
                    penalty += (self.fisher[n] * (p - self.params[n]).pow(2)).sum()
            return self.ewc_lambda * penalty

        elif self.technique == "lwf" and self.old_model:
            with torch.no_grad():
                logits = inputs['logits']
                actual_inputs = {k:torch.squeeze(v).to(self.device) for k,v in inputs.items() if k != "logits"}
                old_outputs = self.old_model(**actual_inputs)
            return self.lwf_lambda * KLDivLoss(reduction='batchmean')(
                torch.log_softmax(logits/self.temperature, dim=1),
                torch.softmax(old_outputs.logits/self.temperature, dim=1)
            ) * (self.temperature ** 2)

        elif self.technique == "mas":
            penalty = 0
            for n, p in self.model.named_parameters():
                if p.requires_grad:
                    penalty += (self.importance[n] * (p - self.old_params[n]).pow(2)).sum()
            return self.mas_lambda * penalty

        return 0

    def pre_backward(self, inputs=None):
        """Operations before backward pass"""
        if self.technique == "agem" and self.memory:
            # Store current gradient
            self.model.zero_grad()
            for inputs_mem, labels_mem in self.memory:
                inputs_mem = {k:torch.squeeze(v).to(self.device) for k,v in inputs_mem.items()}
                labels_mem = torch.squeeze(labels_mem).to(self.device)
                outputs = self.model(**inputs_mem)
                loss = loss_f(outputs.logits, labels_mem)
                loss.backward()

            self.ref_grad = [p.grad.clone() for p in self.model.parameters() if p.requires_grad] # i think this should be with req grad
            self.model.zero_grad()

    def post_backward(self):
        """Operations after backward pass"""
        if self.technique == "agem" and hasattr(self, 'ref_grad'):
            # Project gradients
            dot_product = sum(torch.sum(p.grad * g_ref)
                        for p, g_ref in zip(self.model.parameters(), self.ref_grad) if p.requires_grad)
            ref_norm = sum(torch.sum(g_ref * g_ref) for g_ref in self.ref_grad)

            if dot_product < 0:  # Negative interference
                scale = dot_product / (ref_norm + 1e-8)
                for p, g_ref in zip(self.model.parameters(), self.ref_grad):
                    if p.grad is not None and p.requires_grad:
                        p.grad -= scale * g_ref

    def post_task_update(self, dataloader=None):
        """Update after each task"""
        if self.technique == "ewc":
            # Compute Fisher information
            self.model.eval()
            for batch in dataloader:
                self.model.zero_grad()
                inputs = {k:torch.squeeze(v).to(self.device) for k, v in batch.items()
                        if k in ['input_ids', 'attention_mask']}
                labels = batch['labels'].to(self.device)

                # outputs = self.model(**inputs, labels=labels)
                outputs = self.model(**inputs)

                loss = loss_f(outputs.logits, labels)
                loss.backward()

                for n, p in self.model.named_parameters():
                    if p.requires_grad and p.grad is not None:
                        self.fisher[n] += p.grad.pow(2) / len(dataloader)

            # Update stored parameters
            self.params = {n: p.clone().detach()
                        for n, p in self.model.named_parameters()
                        if p.requires_grad}

        elif self.technique == "agem":
            # Update memory buffer
            self.memory = []
            for batch in dataloader:
                inputs = {k: torch.squeeze(v).to(self.device) for k, v in batch.items()
                        if k in ['input_ids', 'attention_mask']}
                labels = torch.squeeze(batch['labels']).to(self.device)
                self.memory.append((inputs, labels))
                if len(self.memory) >= self.mem_size:
                    break

        elif self.technique == "lwf":
            # Save model snapshot
            self.old_model = deepcopy(self.model)
            self.old_model.eval()

        elif self.technique == "mas":
            # Update importance weights
            self.model.eval()
            for batch in dataloader:
                self.model.zero_grad()
                inputs = {k: torch.squeeze(v).to(self.device) for k, v in batch.items()
                        if k in ['input_ids', 'attention_mask']}

                outputs = self.model(**inputs)
                # does this need a loss?????????????
                torch.norm(outputs.logits, p=2, dim=1).mean().backward()

                for n, p in self.model.named_parameters():
                    if p.requires_grad and p.grad is not None:
                        self.importance[n] += p.grad.abs() / len(dataloader)

            # Update stored parameters
            self.old_params = {n: p.clone().detach()
                            for n, p in self.model.named_parameters()
                            if p.requires_grad}

class AutoContinualLearner(nn.Module):
    def __init__(self, model_name, device, quantization_config, torch_dtype=torch.bfloat16):
        super().__init__()
        self.device = device
        # self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            quantization_config=quantization_config,
            torch_dtype=torch_dtype
        )
        self.n_initial_params = sum(t.numel() for t in self.model.parameters())
        self.n_trainable_params_initial = sum(t.numel() for t in self.model.parameters() if t.requires_grad)
        self.cl = None

    def init_cl(self, technique, lora_config, **kwargs):
        """Init the continual learning technique"""
        self.model = get_peft_model(self.model, lora_config).to(self.device)
        self.n_params_lora = sum(t.numel() for t in self.model.parameters())
        self.n_trainable_params_lora = sum(t.numel() for t in self.model.parameters() if t.requires_grad)
        self.model.print_trainable_parameters()
        self.cl = CLTechniques(self.model, self.device, technique, **kwargs)
    def forward(self, **kwargs):
        return self.model(**kwargs)


In [ ]:
cl_technique = "ewc"

if cl_technique in ["ewc", "agem", "lwf", "mas"]:
    cl_hyperparams = {
    "ewc": {"ewc_lambda":1500},
    "agem": {"mem_size":100},
    "lwf": {"lwf_lambda":1,
            "temperature":2},
    "mas": {"mas_lambda":1000}
    }

    cl_params = cl_hyperparams[cl_technique]
    hyper_param_str = "=".join([str(k) + "-" + str(v) for k, v in cl_params.items()])


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
model = AutoContinualLearner(model_id + "/Model", device, bnb_config)
model.init_cl(technique=cl_technique, lora_config=config, **cl_params)

trainable params: 1,638,400 || all params: 363,459,520 || trainable%: 0.4508


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id + "/Tokenizer")
if tokenizer.pad_token is None and "Llama" in model_id: tokenizer.pad_token = '<|finetune_right_pad_id|>'
elif tokenizer.pad_token is None: tokenizer.pad_token = tokenizer.eos_token
tokenizer.chat_template = open(model_id + "/Tokenizer/chat_template.jinja").read()



def preprocess_and_tokenize(clean_post, label, base_prompt=base_prompt, max_length=512):
    
    prompt_plus_messages = base_prompt.format(clean_post)
    messages = [
            {"role": "system", "content": "You are a helpful assistant"},
            {"role": "user", "content": prompt_plus_messages},
            {"role": "assistant", "content": label.strip("\n")}
        ]

    chat_template = tokenizer.apply_chat_template(messages, tokenize=False, continue_final_message=False, add_special_tokens=False).rstrip()
    input_ids_tokenized = tokenizer(chat_template, return_tensors="pt", add_special_tokens=False, padding="max_length", max_length=max_length)["input_ids"]

    # getting the normal text just to know how much we need to add to the left as -100 and right as pad token
    input_ids_shape = tokenizer(chat_template, return_tensors="pt", add_special_tokens=False, padding=False)["input_ids"]

    # getting the label target to only predict the actual label and ignore the prompt
    labels_tokenized = tokenizer(label + tokenizer.eos_token, add_special_tokens=True, return_tensors="pt")["input_ids"]
    shape = input_ids_shape.shape[1] - labels_tokenized.shape[1]
    zeros = torch.zeros((1, shape), dtype=labels_tokenized.dtype, device=labels_tokenized.device)
    zeros.fill_(-100) # for the cross entropy loss
    labels_left_padded = torch.cat([zeros, labels_tokenized], dim=1)

    eos_n = input_ids_tokenized.shape[1] - labels_left_padded.shape[1]
    eos_n_tensor = torch.zeros((1, eos_n), dtype=labels_tokenized.dtype, device=labels_tokenized.device)
    eos_n_tensor.fill_(tokenizer.encode(tokenizer.pad_token, add_special_tokens=False)[0])
    labels_padded = torch.cat([labels_left_padded, eos_n_tensor], dim=1)

    # print(labels_padded.shape == input_ids_tokenized.shape)

    # shifting because we dont predict the first token
    input_ids_tokenized_left_shifted = input_ids_tokenized[:, :-1]
    labels_tokenized_right_shifted = labels_padded[:, 1:]

    attention_mask = input_ids_tokenized_left_shifted != tokenizer.pad_token_id
    
    return {
        "input_ids": input_ids_tokenized_left_shifted,
        "labels": labels_tokenized_right_shifted,
        "attention_mask": attention_mask
    }


In [ ]:
base_prompt = """You are a social media content moderator.
INSTRUCTION: The following is a social media message that needs to be classified with the label HATEFUL or NOT HATEFUL.
MESSAGE: {}
OUTPUT AND FORMAT: your output should be just the label."""


def format_prompt(text, base_prompt=base_prompt):

    formatted_prompt = base_prompt.format(text)
    
    return formatted_prompt

def translate_class_to_label(class_):

    translation_dict = {"not_hate": "NOT HATEFUL",
                        "explicit_hate": "HATEFUL",
                        "implicit_hate": "HATEFUL"}

    translated_label = translation_dict[class_]

    return translated_label


In [ ]:
dataset_path = "df_from_exp_to_imp.csv"

In [ ]:
print("----------Preparing the Data-----------------")

print("_________________________________")
print("Loading and filtering the Data")

df = pd.read_csv(dataset_path)

#### Attaching the prompt to the clean post
df["formatted_prompt"] = df["clean_post"].apply(format_prompt)
df["label"] = df["class"].apply(translate_class_to_label)

testing_order = ["explicit_hs"]

----------Preparing the Data-----------------
_________________________________
Loading and filtering the Data


In [ ]:
datasets_test = []
for i, task in enumerate(testing_order):
    print("Task")
    print(task)
    split_df = df[(df["split"] == "test") & (df["task"] == task)]
    hf_split = Dataset.from_pandas(split_df)
    datasets_test.append({testing_order[i]: hf_split})


Task
explicit_hs


In [ ]:
datasets_test

[{'explicit_hs': Dataset({
      features: ['clean_post', 'post', 'class', 'implicit_class', 'extra_implicit_class', 'target', 'implied_statement', 'split', 'time', 'task', 'formatted_prompt', 'label', '__index_level_0__'],
      num_rows: 720
  })}]

In [ ]:
example = datasets_test[0]["explicit_hs"][21]["formatted_prompt"]

In [ ]:
pp(example)

('You are a social media content moderator.\n'
 'INSTRUCTION: The following is a social media message that needs to be '
 'classified with the label HATEFUL or NOT HATEFUL.\n'
 "MESSAGE: does this mean white south africans aren't actually white  because "
 "they're born in pri\n"
 'OUTPUT AND FORMAT: your output should be just the label.')


In [ ]:
messages = [
            {"role": "system", "content": "You are a helpful assistant"},
            {"role": "user", "content": example}
        ]

In [ ]:
chat_template = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
pp(chat_template)

('<|im_start|>system\n'
 'You are a helpful assistant<|im_end|>\n'
 '<|im_start|>user\n'
 'You are a social media content moderator.\n'
 'INSTRUCTION: The following is a social media message that needs to be '
 'classified with the label HATEFUL or NOT HATEFUL.\n'
 "MESSAGE: does this mean white south africans aren't actually white  because "
 "they're born in pri\n"
 'OUTPUT AND FORMAT: your output should be just the label.<|im_end|>\n'
 '<|im_start|>assistant\n')


In [ ]:
input_dict = tokenizer(chat_template, return_tensors="pt", add_special_tokens=False)
input_dict = {k: v.to(device) for k, v in input_dict.items()}
input_ids_tokenized = input_dict["input_ids"]
attention_mask = input_dict["attention_mask"]


In [ ]:
generated_tokens = model.model.generate(input_ids=input_ids_tokenized, 
                                        attention_mask=attention_mask, 
                                        top_p=0.9, 
                                        temperature=0.6, 
                                        max_new_tokens=10,
                                        return_dict_in_generate=False)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [ ]:
seq = generated_tokens[0]
seq

tensor([    1,  9690,   198,  2683,   359,   253,  5356, 11173,     2,   198,
            1,  4093,   198,  2683,   359,   253,  1329,  2902,  2627, 45057,
           30,   198,  2113, 47687,  4171,    42,   378,  1695,   314,   253,
         1329,  2902,  3714,   338,  1923,   288,   325,  9827,   351,   260,
         4368,   407,  9081,    54,  6565,   355,  9695,   407,  9081,    54,
         6565,    30,   198, 26826, 41601,    42,  1072,   451,  1441,  2537,
         4203, 46230,   487,  4775,   982,  2390,  2537,   216,   975,   502,
         2316,  3988,   281, 24993,   198, 44724,  7485, 12111, 40560,    42,
          469,  3124,   868,   325,   915,   260,  4368,    30,     2,   198,
            1,   520,  9531,   198, 18083,   407,  9081,    54,  6565,     2],
       device='cuda:0')

In [ ]:
pred = tokenizer.decode(seq[input_ids_tokenized.shape[1]:], skip_special_tokens=True)

In [ ]:
pp(pred)

'NOT HATEFUL'


In [ ]:
print("_________________________________")
print("Testing the model")

predictions_test = []
labels_test = []
predicted_strings = []
labels_strings = []
full_generation = []

model.eval()
with torch.no_grad():
    # print("TESTING DS")
    # print(ds)
    # print()
    # for i, test_item in enumerate(ds["test"]):
    for i, test_item in enumerate(datasets_test[0].values()):
        # print("TESTING ITEM")
        # print(test_item)
        # print(i)
        target_label = test_item["label"]
        labels_strings.append(target_label)
        # print("TARGET LABEL")
        # print(target_label)
        if target_label == "NOT HATEFUL":
            target_label = 0
        elif target_label == "HATEFUL":
            target_label = 1
        
        labels_test.append(target_label)

        formatted_prompt = test_item["formatted_prompt"]
        # prompt_plus_messages = base_prompt.format(clean_post)
        # print("FORMATTED PROMPT")
        # print(formatted_prompt)


        messages = [
            {"role": "system", "content": "You are a helpful assistant"},
            {"role": "user", "content": formatted_prompt}
        ]
        chat_template = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        # print("CHAT TEMPLATE COMPUTED")
        # print(chat_template)
        input_dict = tokenizer(chat_template, return_tensors="pt", add_special_tokens=False)
        input_dict = {k: v.to(device) for k, v in input_dict.items()}
        input_ids_tokenized = input_dict["input_ids"]
        attention_mask = input_dict["attention_mask"]
        
        # print("TOKENIZED CHAT TEMPLATE COMPUTED")
        # print(input_ids_tokenized)
        # print(type(input_ids_tokenized))
        # if input_ids_tokenized.shape[0] == 1:
        #     print("wrong size")
        #     input_ids_tokenized = input_ids_tokenized.squeeze(0)
        #     attention_mask = attention_mask.squeeze(0)
        # print("NEW SHAPE")
        # print(input_ids_tokenized.shape)
        # print(attention_mask.shape)
        # ######################
        # print("----------------right beforeoutput---------------------------------------")
        # # print(model)
        # # print(model.module)
        # # print(dir(model))
        # # print(dir(model.module))
        # # print(help(model.module.generate))
        # print(model.module.generate(input_ids=input_ids_tokenized, 
        #                                 attention_mask=attention_mask, 
        #                                 top_p=0.9, 
        #                                 temperature=0.6, 
        #                                 max_new_tokens=10,
        #                                 return_dict_in_generate=False))
        # print("----------------right after output---------------------------------------")
        output = model.module.model.generate(input_ids=input_ids_tokenized, 
                                        attention_mask=attention_mask, 
                                        top_p=0.9, 
                                        temperature=0.6, 
                                        max_new_tokens=10,
                                        return_dict_in_generate=False)
                
        # pred = tokenizer.batch_decode(output, skip_special_tokens=True)
        # print("OUTPUT COMPUTED")
        # print(output)
        # print(type(output))
        seq = output[0]
        # print(tokenizer.decode(seq, skip_special_tokens=True).strip())
        pred = tokenizer.decode(seq[input_ids_tokenized.shape[1]:], skip_special_tokens=True)
        print(pred)
        print(seq)
        full_generation.append(pred)
        predicted_strings.append(pred)
        # print("PRED COMPUTED")
        # print(pred)
        pred_label = translate_prediction_to_label(pred)
        # print("PRED LABEL COMPUTED")
        # print(pred_label)
        predictions_test.append(pred_label)


_________________________________
Testing the model


TypeError: can only concatenate str (not "list") to str

In [ ]:


# ### Turning the Df into a DatasetDict


times_array = list(df["time"].unique())
datasets = []
dataset_names = list(df["task"].unique())

for time in times_array:

    time_ds = []
    for split in df["split"].unique():

        split_df = df[(df["split"] == split) & (df["time"] == time)]
        hf_split = Dataset.from_pandas(split_df)
        time_ds.append(hf_split)
    datasets.append(time_ds)

hf_datasets = []

for i, dataset in enumerate(datasets):

    hf_ds = DatasetDict({dataset[0]["split"][0]: dataset[0], 
                        dataset[1]["split"][0]: dataset[1],
                        dataset[2]["split"][0]: dataset[2]})
    hf_ds_name = dataset_names[i]
    hf_datasets.append({hf_ds_name: hf_ds})

hf_datasets = [
    {task_name: hf_time.map(preprocess_and_tokenize, input_columns=["clean_post", "label"], batched=False)}
    for hf_data in hf_datasets
    for task_name, hf_time in hf_data.items() 
]

n_samples_per_ds = [
    len(hf_time["train"])
    for hf_data in hf_datasets
    for task_name, hf_time in hf_data.items() 
]

for ds in hf_datasets:
    for hf_data in ds.values():
        hf_data.set_format("torch")

cols_to_remove = ["clean_post", "post", "class", "implicit_class", "extra_implicit_class", 
                "target", "implied_statement", "split", "time", "task",
                "formatted_prompt", "label", "__index_level_0__"]

hf_datasets = [
    {task_name: {split: hf_time[split].remove_columns(cols_to_remove)}}
    for hf_data in hf_datasets
    for task_name, hf_time in hf_data.items()
    for split in hf_time 
    if split != "test"]

print("hf_datasets before data collator:")
print(hf_datasets)
print()
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# distributed_samplers = [
#     {task_name: {split: DistributedSampler(hf_time[split], num_replicas=world_size, rank=local_rank, shuffle=False)}}
#     for hf_data in hf_datasets
#     for task_name, hf_time in hf_data.items()
#     for split in hf_time 
#     if split != "test"
# ]

distributed_samplers = []
for ds in hf_datasets:
    ds_dict = {}
    print("ds:")
    print(ds)
    for task_name, hf_data in ds.items():
        print("task_name:")
        print(task_name)
        print("hf_data:")
        print(hf_data)
        ds_dict[task_name] = {}
        for split in hf_data:
            print("split:")
            print(split)
            if split != "test":
                distr_sampler = DistributedSampler(hf_data[split], num_replicas=world_size, rank=local_rank, shuffle=False)
                ds_dict[task_name][split] = distr_sampler
        dsitr_samplers.append(ds_dict)

data_loaders = []
for i, distr_sampler in enumerate(distributed_samplers):
    ds_name = list(distr_sampler.keys())[0]
    ds_dict = {}
    ds_dict[ds_name] = {}
    for split, distributed_sampler in distr_sampler[ds_name].items():
        data_loader = DataLoader(hf_datasets[i][ds_name][split], collate_fn=data_collator, batch_size=batch_size, sampler=distributed_sampler)
        ds_dict[ds_name][split] = data_loader
    data_loaders.append(ds_dict)

c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\distributed\distributed_c10d.py:750: UserWarning: Attempted to get default timeout for nccl backend, but NCCL support is not compiled
  warnings.warn(


ValueError: Default process group has not been initialized, please make sure to call init_process_group.

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

distributed_samplers = [
    {task_name: {split: DistributedSampler(hf_time[split], num_replicas=world_size, rank=local_rank, shuffle=False)}}
    for hf_data in hf_datasets
    for task_name, hf_time in hf_data.items()
    for split in hf_time 
    if split != "test"
]

data_loaders = []
for i, distr_sampler in enumerate(distributed_samplers):
    ds_name = list(distr_sampler.keys())[0]
    ds_dict = {}
    ds_dict[ds_name] = {}
    for split, distributed_sampler in distr_sampler[ds_name].items():
        data_loader = DataLoader(hf_datasets[i][ds_name][split], collate_fn=data_collator, batch_size=batch_size, sampler=distributed_sampler)
        ds_dict[ds_name][split] = data_loader
    data_loaders.append(ds_dict)

# data loader = []
# each item in the list is a dictionary of {<dataset_name>: {<split>: <dataloade>}}



## Model Stuff

In [ ]:
bnb_config = BitsAndBytesConfig(  
                                load_in_4bit= True,
                                bnb_4bit_quant_type= "nf4",
                                bnb_4bit_compute_dtype= torch.bfloat16,
                                bnb_4bit_use_double_quant= True,
                            )


In [ ]:
model_id = "Models/Llama-3.2-1B-Instruct/Model"
tokenizer_id = "Models/Llama-3.2-1B-Instruct/Tokenizer"

model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(tokenizer_id)

# model.save_pretrained("Models/Llama-3.2-1B-Instruct/Model")
# tokenizer.save_pretrained("Models/Llama-3.2-1B-Instruct/Tokenizer")

Some parameters are on the meta device because they were offloaded to the cpu.


In [ ]:
warnings.filterwarnings("ignore") 
# log_hf()
load_dotenv("env_vars.env")

set_seed(42)
random.seed(42)
torch.manual_seed(42)
np.random.seed(42)


In [ ]:
batch_size = 8
n_epochs = 2
lr = 1e-5
lora_r = 8

In [ ]:
########################################################## DATA WORK
print("_________________________________")
print("Preapring the Data")


df = pd.read_csv("df_from_exp_to_imp.csv")

base_prompt = """You are a social media content moderator.
INSTRUCTION: The following is a social media message that needs to be classified with the label HATEFUL or NOT HATEFUL.
MESSAGE: {}
OUTPUT AND FORMAT: your output should be just the label."""

_________________________________
Preapring the Data


In [ ]:
tokenizer.SPECIAL_TOKENS_ATTRIBUTES

['bos_token',
 'eos_token',
 'unk_token',
 'sep_token',
 'pad_token',
 'cls_token',
 'mask_token',
 'additional_special_tokens']

In [ ]:
# tokenizer.encode(tokenizer.pad_token)

In [ ]:
print(tokenizer.mask_token)

None


In [ ]:
print(tokenizer.pad_token)
if tokenizer.pad_token is None: tokenizer.pad_token = '<|finetune_right_pad_id|>'

None


In [ ]:
print(tokenizer.pad_token)
print(type(tokenizer.encode(tokenizer.pad_token, add_special_tokens=False))[0])

<|finetune_right_pad_id|>
list[0]


In [ ]:
tokenizer.encode(tokenizer.pad_token, add_special_tokens=True)[0]

128000

In [ ]:
tokenizer.encode(tokenizer.pad_token, add_special_tokens=False)[0]

128004

In [ ]:
tokens = tokenizer.encode("Hello, how are you?", return_tensors="pt")
print(tokens.shape)
second = tokens.fill_(-100)
tok = tokenizer.encode("Hello, how are you?", return_tensors="pt")

# torch_tensor = torch.tensor(tokens)


torch.Size([1, 7])


In [ ]:
both = torch.cat((tokens,tok), dim=1)
both

tensor([[  -100,   -100,   -100,   -100,   -100,   -100,   -100, 128000,   9906,
             11,   1268,    527,    499,     30]])

In [ ]:
second

tensor([[-100, -100, -100, -100, -100, -100, -100]])

In [ ]:
# out = model(both)
# print(out)

In [ ]:
tokenizer.pad_token

'<|finetune_right_pad_id|>'

In [ ]:
def translate_class_to_label(class_):

    translation_dict = {"not_hate": "NOT HATEFUL",
                        "explicit_hate": "HATEFUL",
                        "implicit_hate": "HATEFUL"}

    translated_label = translation_dict[class_]

    return translated_label


In [ ]:
def format_message(formatted_prompt, label=True):
    if label:
        messages = [
            {"role": "system", "content": "You are a helpful assistant"},
            {"role": "user", "content": formatted_prompt},
            {"role": "assistant", "content": label}
        ]
    else:
        messages = [
            {"role": "system", "content": "You are a helpful assistant"},
            {"role": "user", "content": formatted_prompt}
        ]
    return messages

def format_prompt(text, base_prompt=base_prompt):

    formatted_prompt = base_prompt.format(text)
    
    return formatted_prompt


In [ ]:
def preprocess_and_tokenize(clean_post, label, base_prompt=base_prompt, max_length=312):

    # if type(label) != list:
    #     label = [label]
    # if type(clean_post) != list:
    #     clean_post = [clean_post]
    
    prompt_plus_messages = base_prompt.format(clean_post)
    # pp(prompt_plus_messages)
    # pp(label)
    messages = [
            {"role": "system", "content": "You are a helpful assistant"},
            {"role": "user", "content": prompt_plus_messages},
            {"role": "assistant", "content": label.strip("\n")}
        ]

    # print(messages)
    chat_template = tokenizer.apply_chat_template(messages, tokenize=False, continue_final_message=False, add_special_tokens=False).rstrip()
    # print(chat_template)

    # why is the chat template putting a new line at the end of the end of sequence
    # pp(chat_template)
    input_ids_tokenized = tokenizer(chat_template, return_tensors="pt", add_special_tokens=False, padding="max_length", max_length=max_length)["input_ids"]

    # getting the normal text just to know how much we need to add to the left as -100 and right as pad token
    input_ids_shape = tokenizer(chat_template, return_tensors="pt", add_special_tokens=False, padding=False)["input_ids"]
    # print(input_ids_tokenized)

    # getting the label target to only predict the actual label and ignore the prompt
    labels_tokenized = tokenizer(label + tokenizer.eos_token, add_special_tokens=True, return_tensors="pt")["input_ids"]
    shape = input_ids_shape.shape[1] - labels_tokenized.shape[1]
    zeros = torch.zeros((1, shape), dtype=labels_tokenized.dtype, device=labels_tokenized.device)
    zeros.fill_(-100) # acc to llama docs
    labels_left_padded = torch.cat([zeros, labels_tokenized], dim=1)

    eos_n = input_ids_tokenized.shape[1] - labels_left_padded.shape[1]
    eos_n_tensor = torch.zeros((1, eos_n), dtype=labels_tokenized.dtype, device=labels_tokenized.device)
    print("FILLING PAD WITH")
    print(tokenizer.encode(tokenizer.pad_token, add_special_tokens=False)[0])
    eos_n_tensor.fill_(tokenizer.encode(tokenizer.pad_token, add_special_tokens=False)[0])
    labels_padded = torch.cat([labels_left_padded, eos_n_tensor], dim=1)

    # print(labels_padded.shape == input_ids_tokenized.shape)

    # shifting because we dont predict the first token
    input_ids_tokenized_left_shifted = input_ids_tokenized[:, :-1]
    labels_tokenized_right_shifted = labels_padded[:, 1:]

    attention_mask = input_ids_tokenized_left_shifted != tokenizer.pad_token_id
    
    return {
        "input_ids": input_ids_tokenized_left_shifted,
        "labels": labels_tokenized_right_shifted,
        "attention_mask": attention_mask
    }


In [ ]:
def loss_f(logits, labels):

    loss_fn = CrossEntropyLoss(reduce=False)
    loss = loss_fn(logits.view(-1, logits.size(-1)), labels.view(-1))
    
    return loss


In [ ]:
   

#### Attaching the prompt to the clean post

df["formatted_prompt"] = df["clean_post"].apply(format_prompt)
df["label"] = df["class"].apply(translate_class_to_label)

# ### Turning the Df into a DatasetDict

t_1 = []
t_2 = []

for split in df["split"].unique():

    split_df_1 = df[(df["split"] == split) & (df["time"] == 1)]
    split_df_2 = df[(df["split"] == split) & (df["time"] == 2)]

    hf_split_1 = Dataset.from_pandas(split_df_1)
    hf_split_2 = Dataset.from_pandas(split_df_2)
    
    t_1.append(hf_split_1)
    t_2.append(hf_split_2)

hf_time_1 = DatasetDict({t_1[0]["split"][0]: t_1[0], 
                        t_1[1]["split"][0]: t_1[1],
                        t_1[2]["split"][0]: t_1[2]})

hf_time_2 = DatasetDict({t_2[0]["split"][0]: t_2[0], 
                        t_2[1]["split"][0]: t_2[1],
                        t_2[2]["split"][0]: t_2[2]})




In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
ex = hf_time_1["train"][0]
input_model = preprocess_and_tokenize(ex["clean_post"], ex["label"], base_prompt=base_prompt, max_length=312)
input_model["labels"]
input_model["input_ids"]

In [ ]:
output_model = model(**input_model)

In [ ]:
output_model.loss

tensor(9.3625, grad_fn=<ToCopyBackward0>)

In [ ]:
output_model.logits.shape

torch.Size([1, 311, 128256])

In [ ]:
input_model["labels"].shape

torch.Size([1, 311])

In [ ]:
logits = output_model.logits
labels = input_model["labels"]

flat_logits = logits.view(-1, logits.size(-1))
flat_labels = labels.view(-1)

In [ ]:
flat_logits.shape

torch.Size([311, 128256])

In [ ]:
flat_labels.shape

torch.Size([311])

In [ ]:
loss_fn = CrossEntropyLoss(ignore_index=-100)  
loss = loss_fn(flat_logits, flat_labels)          
loss

tensor(8.9375, dtype=torch.bfloat16, grad_fn=<NllLossBackward0>)

In [ ]:
labels.view(-1).shape

torch.Size([311])

Checking that the -100 is not being computed!!

In [ ]:
mask = (flat_labels != -100)
mask.sum().item()

207

In [ ]:
valid_logits = flat_logits[mask]   
valid_labels = flat_labels[mask]        

In [ ]:
manual_loss = loss_fn(valid_logits, valid_labels)
manual_loss

tensor(8.9375, dtype=torch.bfloat16, grad_fn=<NllLossBackward0>)

In [ ]:
def loss_f(logits, labels):

    loss_fn = CrossEntropyLoss(reduce=False)
    loss = loss_fn(logits.view(-1, logits.size(-1)), labels.view(-1))
    
    return loss


In [ ]:
########################################################## TOKENIZER WORK

hf_time_1 = hf_time_1.map(preprocess_and_tokenize, input_columns=["formatted_prompt", "label"], batched=False)
hf_time_2 = hf_time_2.map(preprocess_and_tokenize, input_columns=["formatted_prompt", "label"], batched=False)



Map: 100%|██████████| 1020/1020 [00:00<00:00, 1156.90 examples/s]


In [ ]:
hf_time_1["train"][0]["label"]

'NOT HATEFUL'

In [ ]:
tokenizer.eos_token

'<|endoftext|>'

In [ ]:
tokenizer.pad_token_id

151643

In [ ]:
tokenizer.decode(151643)
end_of_text_token = 151643

In [ ]:
hate_encoding = tokenizer.encode("HATEFUL")


In [ ]:
not_hate_encoding = tokenizer.encode("NOT HATEFUL")


In [ ]:
example = hf_time_1["train"][0]
example.keys()

dict_keys(['clean_post', 'post', 'class', 'implicit_class', 'extra_implicit_class', 'target', 'implied_statement', 'split', 'time', 'formatted_prompt', 'label', '__index_level_0__', 'input_ids', 'attention_mask'])

In [ ]:
end_prompt = example["input_ids"][0].index(end_of_text_token)
end_prompt

90

In [ ]:
label_encoding = tokenizer.encode(example["label"] + tokenizer.eos_token)
label_encoding

[14065, 472, 2336, 49636, 151643]

In [ ]:
tokenizer.decode(14065)

'NOT'

In [ ]:
start_answer = end_prompt-(len(label_encoding)+1)

In [ ]:
tokenizer.decode(example["input_ids"][0][: start_answer])

In [ ]:
tokenizer.decode(example["input_ids"][0][start_answer : end_prompt - 1])

'NOT HATEFUL<|im_end|>'

In [ ]:
example.keys()

dict_keys(['clean_post', 'post', 'class', 'implicit_class', 'extra_implicit_class', 'target', 'implied_statement', 'split', 'time', 'formatted_prompt', 'label', '__index_level_0__', 'input_ids', 'attention_mask'])

In [ ]:
# base_prompt



In [ ]:
tokenizer.decode(198)

'\n'

In [ ]:
if tokenizer.pad_token is None: tokenizer.pad_token = '<|finetune_right_pad_id|>'

def preprocess_and_tokenize(clean_post, label, base_prompt=base_prompt, max_length=312):

    # if type(label) != list:
    #     label = [label]
    # if type(clean_post) != list:
    #     clean_post = [clean_post]
    
    prompt_plus_messages = base_prompt.format(clean_post)
    # pp(prompt_plus_messages)
    # pp(label)
    messages = [
            {"role": "system", "content": "You are a helpful assistant"},
            {"role": "user", "content": prompt_plus_messages},
            {"role": "assistant", "content": label.strip("\n")}
        ]

    # print(messages)
    chat_template = tokenizer.apply_chat_template(messages, tokenize=False, continue_final_message=False, add_special_tokens=False).rstrip()
    # print(chat_template)

    # why is the chat template putting a new line at the end of the end of sequence
    # pp(chat_template)
    input_ids_tokenized = tokenizer(chat_template, return_tensors="pt", add_special_tokens=False, padding="max_length", max_length=max_length)["input_ids"]

    # getting the normal text just to know how much we need to add to the left as -100 and right as pad token
    input_ids_shape = tokenizer(chat_template, return_tensors="pt", add_special_tokens=False, padding=False)["input_ids"]
    # print(input_ids_tokenized)

    # getting the label target to only predict the actual label and ignore the prompt
    labels_tokenized = tokenizer(label + tokenizer.eos_token, add_special_tokens=True, return_tensors="pt")["input_ids"]
    shape = input_ids_shape.shape[1] - labels_tokenized.shape[1]
    zeros = torch.zeros((1, shape), dtype=labels_tokenized.dtype, device=labels_tokenized.device)
    zeros.fill_(-100) # acc to llama docs
    labels_left_padded = torch.cat([zeros, labels_tokenized], dim=1)

    eos_n = input_ids_tokenized.shape[1] - labels_left_padded.shape[1]
    eos_n_tensor = torch.zeros((1, eos_n), dtype=labels_tokenized.dtype, device=labels_tokenized.device)
    eos_n_tensor.fill_(tokenizer.eos_token_id)
    labels_padded = torch.cat([labels_left_padded, eos_n_tensor], dim=1)

    # print(labels_padded.shape == input_ids_tokenized.shape)

    # shifting because we dont predict the first token
    input_ids_tokenized_left_shifted = input_ids_tokenized[:, :-1]
    labels_tokenized_right_shifted = labels_padded[:, 1:]

    attention_mask = input_ids_tokenized_left_shifted != tokenizer.pad_token_id
    
    return {
        "input_ids": input_ids_tokenized_left_shifted,
        "labels": labels_tokenized_right_shifted,
        "attention_mask": attention_mask
    }

In [ ]:
hf_time_1 = hf_time_1.map(preprocess_and_tokenize, input_columns=["clean_post", "label"], batched=False)


Map: 100%|██████████| 720/720 [00:01<00:00, 622.72 examples/s]


In [ ]:
hf_time_1.set_format("torch")
hf_time_2.set_format("torch")

cols_to_remove = ["clean_post", "post", "class", "implicit_class", "extra_implicit_class", "target", "implied_statement", "split", "time", "formatted_prompt", "label", "__index_level_0__"]

for split in hf_time_1:
    if split != "test":
        hf_time_1[split] = hf_time_1[split].remove_columns(cols_to_remove)
        hf_time_2[split] = hf_time_2[split].remove_columns(cols_to_remove)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


hf_time_1_train_loader = DataLoader(hf_time_1["train"], collate_fn=data_collator, batch_size=batch_size)
hf_time_1_validation_loader = DataLoader(hf_time_1["validation"], collate_fn=data_collator, batch_size=batch_size)
hf_time_1_test_loader = DataLoader(hf_time_1["test"], collate_fn=data_collator, batch_size=batch_size)

hf_time_2_train_loader = DataLoader(hf_time_2["train"], collate_fn=data_collator, batch_size=batch_size)
hf_time_2_validation_loader = DataLoader(hf_time_2["validation"], collate_fn=data_collator, batch_size=batch_size)
hf_time_2_test_loader = DataLoader(hf_time_2["test"], collate_fn=data_collator, batch_size=batch_size)

# ### So far, created the prompt, did the messages with the prompt and answer in place. Applied to chat template and tokenized 


In [ ]:
########################3#################### MODEL WORK

print("_________________________________")
print("Loading the model and model config")

bnb_config = BitsAndBytesConfig(  
                                load_in_4bit= True,
                                bnb_4bit_quant_type= "nf4",
                                bnb_4bit_compute_dtype= torch.bfloat16,
                                bnb_4bit_use_double_quant= True,
                            )

model = AutoModelForCausalLM.from_pretrained(model_id,
                                            torch_dtype=torch.bfloat16,
                                            device_map="auto",
                                            quantization_config=bnb_config
                                            )

# to deal with the fact that we dont make the first token prediction??


model_size_before = sum(t.numel() for t in model.parameters())
print("Model Size before LoRA", model_size_before)
print(model)
print()

lora_alpha = lora_r*2
config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    task_type="CAUSAL_LM",
    lora_dropout=0.1,
    bias="none",
)

model = get_peft_model(model, config)
print("Model After LoRA")
model.print_trainable_parameters()

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
model.to(device)


loss_fn = CrossEntropyLoss()
optimizer = AdamW((param for param in model.parameters() if param.requires_grad), lr=lr)


_________________________________
Loading the model and model config
Model Size before LoRA 315119488
Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear4bit(in_features=896, out_features=896, bias=True)
          (k_proj): Linear4bit(in_features=896, out_features=128, bias=True)
          (v_proj): Linear4bit(in_features=896, out_features=128, bias=True)
          (o_proj): Linear4bit(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear4bit(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear4bit(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layer

In [ ]:
print("_________________________________")
print("Training the model")
print()

for epoch in range(n_epochs):

    torch.cuda.empty_cache()
    gc.collect()
    model.train()

    print("Epoch: ", epoch)
    losses = []

    for i, batch in enumerate(hf_time_1_train_loader):
        if i > 0:
            continue

        torch.cuda.empty_cache()
        gc.collect()

        print("\tBatch: ", i)
        # print(batch)
        batch.to(device)
        # print(batch.keys())
        # print(batch["input_ids"].shape)
        # print(batch["attention_mask"].shape)
        # print(batch["labels"].shape)


        batch = {k:torch.squeeze(v) for k,v in batch.items()}

        # print(batch["input_ids"].shape)
        # print(batch["attention_mask"].shape)
        # print(batch["labels"].shape)


        output = model(**batch)
        logits = output.logits
        loss = loss_fn(logits, batch["labels"])

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        losses.append(loss.detach().item())

        print(batch.keys())
        print(loss.detach().item())
        print(output.logits.shape)
        print(output.probas)

        if i > 3:
            continue

    epoch_loss = sum(losses)/len(hf_time_1_train_loader)
    print(f"Epoch {epoch} Loss: {epoch_loss}")

    model.eval()
    with torch.no_grad():  

        torch.cuda.empty_cache()
        gc.collect()

        val_losses = []

        for i, batch in enumerate(hf_time_1_validation_loader):
            if i > 0:
                continue
            batch.to(device)
            batch = {k:torch.squeeze(v) for k,v in batch.items()}

            output = model(**batch)
            logits = output.logits
            val_loss = loss_fn(logits, batch["labels"])

            val_losses.append(val_loss.detach().item())

        val_loss_epoch = sum(val_losses)/len(hf_time_1_validation_loader)
        print(f"Epoch {epoch} Validation Loss: {val_loss_epoch}")
print()


In [ ]:
print("_________________________________")
print("Testing the model")
for i, test_batch in enumerate(hf_time_1["test"]):

    if i > 0:
        break
    
    text = test_batch["formatted_prompt"]
    tokenized_chat_template, messages_list = preprocess_and_tokenize(text, label=False, add_generation_prompt=True, output_messages_list=True)
    output = model.generate(**tokenized_chat_template.to(device))
    pred = tokenizer.decode(output[0], skip_special_tokens=True)
    
    print(text)
    print(tokenized_chat_template)
    print(output)
    print(pred)

print("CHECKING GENERATION")
print(messages_list)

print(tokenized_chat_template)
print(output)

print(type(output))
print(output.shape)

print("_________________________________")
print("Saving the model and Tokenizer")
model_name = model_id.split("/")[-1]
model.save_pretrained(f"alberto-lorente/{model_name}_test")
tokenizer.save_pretrained(f"alberto-lorente/{model_name}_test")

print("RUN SUCCESSFULLY")
